Loading the necessary packages

In [ ]:
from __future__ import print_function

from depmapomics.config import *

from depmapomics import tracker, loading, fusions, expressions
from genepy import terra
from genepy.utils import helper as h

import dalmatian as dm
import seaborn as sns

from bokeh.plotting import output_notebook

%load_ext autoreload
%autoreload 2

output_notebook()

immutable parameters (user specific)

# Generate sample set from new samples

we retrieve all the samples we can find from the GP workspaces

__CCLE specific__

In [ ]:
CCLEref

In [ ]:
l = CCLEref[CCLEref.source.isna() & (CCLEref.datatype=="rna")].arxspan_id

In [ ]:
l

In [ ]:
h.dups(l.values)

In [ ]:
a = pv.loc[l.values,'Flagship'].str.contains('IBM')

In [ ]:
dm.WorkspaceManager(rnawo)

In [ ]:
a.loc[a[a.isna()].index.tolist()]=False

In [ ]:
a

In [ ]:
v

In [ ]:
a.loc[val]

In [ ]:
CCLEref.loc[l, 'Flagship']

In [ ]:
CCLEref = CCLEref.drop(columns="Flagship")

In [ ]:
CCLEref.

In [ ]:
for k, val in l.iteritems():
    CCLEref.loc[k, 'source']= 'IBM' if a.loc[val] else 'DEPMAP'

In [ ]:
CCLEref.loc[l.index, 'source']

In [ ]:
if isCCLE:
    print("loading new RNAseq data")
    samples = loading.loadRNA(SAMPLESETNAME)

In [ ]:
if isCCLE:
    print("uploading samples to the tracker and Terra")
    loading.update(samples, samplesetname=SAMPLESETNAME, stype='rna', bucket="gs://cclebams/rna/", refworkspace=RNAWORKSPACE)

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 6 functions To generate the expression/fusion dataset:

We use the GTEx pipeline ([https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md](https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md)).

To generate the expression dataset, run the following tasks on all samples that you need, in this order:



*   samtofastq_v1-0_BETA_cfg 

    (broadinstitute_gtex/samtofastq_v1-0_BETA Snapshot ID: 5)

*   star_v1-0_BETA_cfg

(broadinstitute_gtex/star_v1-0_BETA Snapshot ID: 7)



*   rsem_v1-0_BETA_cfg 

    (broadinstitute_gtex/rsem_v1-0_BETA Snapshot ID: 4)

*   rsem_aggregate_results_v1-0_BETA_cfg (broadinstitute_gtex/rsem_aggregate_results_v1-0_BETA Snapshot ID: 3)

The outputs to be downloaded will be saved under the sample set that you ran. The outputs we use for the release are:



*   rsem_genes_expected_count
*   rsem_genes_tpm
*   rsem_transcripts_tpm

****Make sure that you delete the intermediate files. These files are quite large so cost a lot to store. To delete, you can either write a task that deletes them or use gsutil rm*****


##### Fusions {#fusions}

We use STAR-Fusion [https://github.com/STAR-Fusion/STAR-Fusion/wiki](https://github.com/STAR-Fusion/STAR-Fusion/wiki). The fusions are generated by running the following tasks



*   hg38_STAR_fusion (gkugener/STAR_fusion Snapshot ID: 14)
*   Aggregate_Fusion_Calls (gkugener/Aggregate_files_set Snapshot ID: 2)

The outputs to be downloaded will be saved under the sample set you ran. The outputs we use for the release are: 



*   fusions_star

This task uses the same samtofastq_v1-0_BETA_cfg task as in the expression pipeline, although in the current implementation, this task will be run twice. It might be worth combing the expression/fusion calling into a single workflow. This task also contains a flag that lets you specify if you want to delete the intermediates (fastqs). 

There are several other tasks in this workspace. In brief:



*   Tasks prefixed with **EXPENSIVE** or **CHEAP** are identical to their non-prefixed version, except that they specify different memory, disk space, etc. parameters. These versions can be used when samples fail the normal version of the task due to memory errors.
*   The following tasks are part of the GTEx pipeline but we do not use them (we use RSEM exclusively): markduplicates_v1-0_BETA_cfg (broadinstitute_gtex/markduplicates_v1-0_BETA Snapshot ID: 2), rnaseqc2_v1-0_BETA_cfg (broadinstitute_gtex/rnaseqc2_v1-0_BETA Snapshot ID: 2)
*   **ExonUsage_hg38_fixed** (gkugener/ExonUsage_fixed Snapshot ID: 1): this task calculates exon usage ratios. The non-fixed version contains a bug in the script that is not able to handle chromosome values prefixed with ‘chr’. The ‘fixed’ version resolves this issue.
*   **AggregateExonUsageRObj_hg38** (ccle_mg/AggregateExonUsageRObj Snapshot ID: 2): combines the exon usage ratios into a matrices that are saved in an R object.

## cleaning workspaces

In [ ]:
if doCleanup:
    print("cleaning workspaces")
    torm = asyncio.run(terra.deleteHeavyFiles(RNAWORKSPACE))
    h.parrun(['gsutil rm '+i for i in torm], cores=8)
    terra.removeFromFailedWorkflows(RNAWORKSPACE, dryrun=False, everythingFor=[])

## On Terra

In [ ]:
# TODO: update with latest workspace parameters from our repo

In [ ]:
print("running Terra pipeline")
refwm = dm.WorkspaceManager(RNAWORKSPACE)
submission_id = refwm.create_submission("RNA_pipeline", SAMPLESETNAME,'sample_set',expression='this.samples')
asyncio.run(terra.waitForSubmission(RNAWORKSPACE, submission_id))

In [ ]:
submission_id = refwm.create_submission("RNA_aggregate", 'all')
asyncio.run(terra.waitForSubmission(RNAWORKSPACE, submission_id))

### Save the workflow configurations used

In [ ]:
terra.saveWorkspace(RNAWORKSPACE,'data/'+SAMPLESETNAME+'/RNAconfig/')

## On Local

### Expression post processing

In [ ]:
expressions.CCLEPostProcessing(samplesetname=SAMPLESETNAME)

### Fusion post processing

In [ ]:
fusions.CCLEPostProcessing(samplesetname=SAMPLESETNAME)